# LangGraph Mini-Lab 1 — Conditional Graph (If/Else Routing)

**Author:** Dr. Dasha Trofimova

Learn to route to different nodes via a conditional edge.

In [ ]:
!pip install -q langgraph langchain langchain-core

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import re

class State(TypedDict):
    query: str
    result: str

def calculator(text: str) -> str:
    if not re.fullmatch(r"[\d\s+\-*/().]+", text):
        return "Calculator only supports + - * / and numbers."
    try:
        return str(eval(text, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Calc error: {e}"

def echo(text: str) -> str:
    return f"Echo: {text}"

def decide(state: State) -> State:
    q = state["query"]
    if any(op in q for op in ["+","-","*","/"]):
        return {"result": "calc"}
    return {"result": "echo"}

def run_calc(state: State) -> State:
    return {"result": calculator(state["query"])}

def run_echo(state: State) -> State:
    return {"result": echo(state["query"])}

def where_next(state: State):
    return state["result"]

g = StateGraph(State)
g.add_node("decide", decide)
g.add_node("calc", run_calc)
g.add_node("echo", run_echo)
g.add_conditional_edges("decide", where_next, {"calc": "calc", "echo": "echo"})
g.add_edge("calc", END)
g.add_edge("echo", END)
g.set_entry_point("decide")

app = g.compile()
print(app.invoke({"query": "2+2"}))
print(app.invoke({"query": "hello"}))

### 🔍 Graph Visualization
Below is a simple directed graph of the nodes and conditional routing.

In [ ]:
!apt-get -qq update && apt-get -qq install -y graphviz > /dev/null
!pip install -q graphviz

In [ ]:
from graphviz import Digraph
gdot = Digraph(comment="Conditional Graph", format="png")
gdot.attr(rankdir="LR", bgcolor="white")
gdot.node("decide", "decide()", shape="diamond", style="rounded,filled", fillcolor="#F3F4F6")
gdot.node("calc", "run_calc()", shape="box", style="rounded,filled", fillcolor="#DBEAFE")
gdot.node("echo", "run_echo()", shape="box", style="rounded,filled", fillcolor="#DCFCE7")
gdot.edge("decide", "calc", label="contains + - * /")
gdot.edge("decide", "echo", label="otherwise")
display(gdot)

**Try it:** change the input and re-run the cell that builds the graph.
The router sends arithmetic-looking queries to `run_calc`, else to `run_echo`.


## 🧩 Agent Anatomy — Conditional Graph

**What this "agent" does:**  
Routes your `query` either to a calculator (if it looks like math) or to an echo node.

**State schema:**  
- `query` *(str)* → user input  
- `result` *(str)* → final output or routing tag during decision

**Nodes:**  
- **decide** → checks `query` for `+ - * /` and returns route (`"calc"` or `"echo"`).  
- **calc** (`run_calc`) → evaluates safe arithmetic and returns string result.  
- **echo** (`run_echo`) → returns `Echo: <query>`.

We'll rebuild the graph with **node descriptions** and a tiny **trace** so you can see what fires.


In [ ]:

from langgraph.graph import StateGraph, END
from typing import TypedDict
import re

class State(TypedDict):
    query: str
    result: str

node_desc = {
    "decide": "Router: inspects the query and decides the next node ('calc' or 'echo').",
    "calc":   "Calculator: evaluates + - * / expressions safely.",
    "echo":   "Echo: returns the original query prefixed with 'Echo:'."
}

def trace(name):
    def deco(fn):
        def wrapped(state: State):
            print(f"\n▶ Node: {name} — {node_desc[name]}")
            out = fn(state)
            print(f"   Input state: {state}")
            print(f"   Output: {out}")
            return out
        wrapped.__doc__ = node_desc[name]
        return wrapped
    return deco

@trace("decide")
def decide(state: State) -> State:
    q = state["query"]
    route = "calc" if any(op in q for op in ["+","-","*","/"]) else "echo"
    return {"result": route}

def calculator(text: str) -> str:
    if not re.fullmatch(r"[\\d\\s+\\-*/().]+", text):
        return "Calculator only supports + - * / and numbers."
    try:
        return str(eval(text, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Calc error: {e}"

@trace("calc")
def run_calc(state: State) -> State:
    return {"result": calculator(state["query"])}

@trace("echo")
def run_echo(state: State) -> State:
    return {"result": f"Echo: {state['query']}"}

def router(state: State):
    return state["result"]

g = StateGraph(State)
g.add_node("decide", decide)
g.add_node("calc", run_calc)
g.add_node("echo", run_echo)
g.add_conditional_edges("decide", router, {"calc": "calc", "echo": "echo"})
g.add_edge("calc", END); g.add_edge("echo", END)
g.set_entry_point("decide")

app = g.compile()
print("\n=== Trace: math query ===")
print(app.invoke({"query": "12*(3+4)", "result": ""}))
print("\n=== Trace: non-math query ===")
print(app.invoke({"query": "hello class", "result": ""}))
